# 멋진 작사가 만들기
## 코드

In [3]:
import os
import re
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# 1. 데이터 읽어오기
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)  # glob.glob() : 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
raw_corpus = []

# txt 파일을 모두 읽어서 한 줄 씩 raw_corpus 에 담음
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("총: ", len(raw_corpus))  # 187088


# 2. 데이터 정제
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                   # 소문자, 양쪽공백 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()                           # 양쪽 공백 제거
    sentence = '<start> ' + sentence + ' <end>'
    return sentence


corpus = []  # 형태 : ['<start> i m begging of you please don t take my man <end>', ...] length - 175986
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    tmp = preprocess_sentence(sentence)
    if len(tmp.split()) > 15: continue
    corpus.append(tmp)

def tokenize(corpus):
    # num_words:전체 단어의 개수, filters:별도로 전처리 로직을 추가, oov_token: out-of-vocabulary 사전에 없었던 단어는 어떤 토큰으로 대체할지
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)  # corpus로부터 Tokenizer가 사전을 자동구축

    # tokenizer를 활용하여 모델에 입력할 데이터셋 구축(Tensor로 변환)
    tensor = tokenizer.texts_to_sequences(corpus)

    # 입력 데이터 시퀀스 길이 맞춰주기 - padding
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)

    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

# 단어 사전 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 5: break

src_input = tensor[:, :-1]  # 소스문장, tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
tgt_input = tensor[:, 1:]   # 타겟문장, tensor에서 <start>를 잘라내서 타겟 문장을 생성 -> 문장 길이는 14가 됨


# 3. 평가데이터셋 분리
# train data를 train, valid로 나눈다.(비율 80:20) 만약 학습데이터 개수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시 검토
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)


# 4. 모델 생성
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)  # 입력된 텐서에는 단어사전의 인덱스가 들어있는데, 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out


embedding_size = 256
hidden_size = 3000
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

# 5. 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=5, validation_data=(enc_val, dec_val))


def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  # 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.

        # 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여줌
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
print(test_sen)

총:  187088
1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
Source Train: (124981, 14)
Target Train: (124981, 14)
Epoch 1/5
3906/3906 [==============================] - 536s 137ms/step - loss: 2.9379 - val_loss: 2.6851
Epoch 2/5
3906/3906 [==============================] - 546s 140ms/step - loss: 2.4189 - val_loss: 2.4374
Epoch 3/5
3906/3906 [==============================] - 532s 136ms/step - loss: 1.9743 - val_loss: 2.2894
Epoch 4/5
3906/3906 [==============================] - 539s 138ms/step - loss: 1.6090 - val_loss: 2.2328
Epoch 5/5
3906/3906 [==============================] - 543s 139ms/step - loss: 1.3606 - val_loss: 2.2440
<start> i love you , i m not gonna crack <end> 


## 회고
### 이번 프로젝트에서 **어려웠던 점**
val_loss를 2.2까지 맞춰야하는데, 생각보다 잘 맞춰지지 않았고, 2.3~2.5를 벗어나기 어려웠다. 하지만 hidden_size를 증가시켰더니 val_loss는 약 2.2정도로 낮아졌다. 이 정도의 변화량이 적당한 것인지 잘 모르겠다.


### 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.
- Embedding(input_dim, output_dim) layer
  - input_dim : vocab_size, 단어 집합의 크기(총 단어의 개수)를 말하며 2D 정수 텐서를 인자로 받는다.
  - output_dim : embedding_size, 임베딩한 후 벡터의 크기로 3D 정수 텐서를 반환한다.  
  
  
- 애매했던 RNN의 개념을 짚고 넘어갈 수 있었다. RNN은 시간의 흐름 혹은 순차적으로 이루어진 시퀀스 데이터 처리에 적합한 모델로, LSTM은 그 중 하나이다. 시퀀스데이터이기 때문에 출력값이 다음의 입력값이 될 수 있다. 기존의 RNN은 문맥상의 흐름을 잘 잡아내지 못하는 장기의존성 문제가 있었는데 이 단점을 해소시켜준 것이 LSTM이다. 기존에는 하나의 내부 레이어로 구성되어 있었지만 LSTM은 4개의 내부 레이어로 구성되어있기 때문이라는데 정확히 이 부분이 어떤 원리로 장기의존성 문제를 해결하는지에 대한 것은 아직 이해하지 못하였다.
  
  
- loss는 정답과의 차이를 나타내는 것이므로 0과 가까울수록 좋은 모델임을 뜻한다.


### 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
epoch 횟수와 embedding_size, hidden_size를 가감하면서 validation loss를 낮춰보려고 했지만, 근사하게만 도달했지 2.2이하로 떨어지진 못했다.

### 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**.
train loss는 0.5-0.3씩 1 epoch당 떨어지는 반면 valid loss는 0.2-0.1씩 떨어지거나 오히려 늘어난 경우도 있었다. 그래서 overfitting도 생각해봤는데, 데이터의 양의 차이가 아닐까 싶어(80:20) 다른 부분을 생각해보았지만, 명확히 떠오르는 것이 없었다.

### 자기 다짐
RNN과 RNN에 해당하는 keras layer종류를 한 번 더 정리할 필요가 있을 것 같다.